In [ ]:
import pandas as pd
import sqlite3 as sql

In [ ]:
conn = sql.connect('/content/drive/MyDrive/PrepinstaPrime/Data Analytics/sql-murder-mystery.db');

In [ ]:
query1 = '''
SELECT *
FROM crime_scene_report
WHERE date = 20180115
AND city = 'SQL City'
AND type = 'murder';
'''
pd.set_option('display.max_colwidth', None);
pd.read_sql_query(query1, conn)

,date,type,description,city
0,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


In [ ]:
query2 = '''
SELECT *
FROM person
WHERE address_street_name = 'Northwestern Dr'
ORDER BY address_number DESC
LIMIT 1;
'''
pd.set_option('display.max_colwidth', None);
pd.read_sql_query(query2, conn, index_col=None)

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [ ]:
query3 = '''
SELECT *
FROM person
WHERE name LIKE 'Annabel%'
AND address_street_name = 'Franklin Ave';
'''

pd.read_sql_query(query3, conn)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [ ]:
query4 = '''
SELECT *
FROM interview
WHERE person_id = 14887;
'''
pd.read_sql_query(query4, conn)


,person_id,transcript
0,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."


In [ ]:
query5 = '''
SELECT *
FROM interview
WHERE person_id = 16371;
'''
pd.read_sql_query(query5, conn)

,person_id,transcript
0,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


When police Intervied witness 1 he told following;

I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".

In [ ]:
query6 = '''
SELECT
    g.id AS gym_id,
    g.person_id,
    g.name,
    p.license_id,
    d.plate_number,
    d.car_make,
    d.car_model,
    g.membership_status
FROM
    get_fit_now_member g
INNER JOIN
    person p ON g.person_id = p.id
INNER JOIN
    drivers_license d ON p.license_id = d.id
WHERE
    g.membership_status = 'gold'
    AND g.id LIKE '48Z%'
    AND d.plate_number LIKE '%H42W%';
'''
pd.read_sql_query(query6, conn)

,gym_id,person_id,name,license_id,plate_number,car_make,car_model,membership_status
0,48Z55,67318,Jeremy Bowers,423327,0H42W2,Chevrolet,Spark LS,gold


When police Intervied witness 2 he told following;

I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th


In [ ]:
query7 = '''
SELECT gc.membership_id as gym_id,  gm.name , gc.check_in_date, gc.check_in_time, gc.check_out_time
FROM get_fit_now_check_in gc
INNER JOIN get_fit_now_member gm ON gc.membership_id = gm.id
WHERE gc.check_in_date = 20180109 and gm.name = "Annabel Miller";
'''
pd.read_sql_query(query7, conn)

,gym_id,name,check_in_date,check_in_time,check_out_time
0,90081,Annabel Miller,20180109,1600,1700


From above query i understood he Annabel Miller checked in at 1600 and checked out at 1700. So Annabel Miller can see the murder between this time.

In [ ]:
query8 = '''
SELECT
    gt.membership_id AS gym_id,
    g.person_id,
    g.name,
    g.membership_status AS status,
	gt.check_in_date AS date,
    gt.check_in_time AS check_in,
    gt.check_out_time AS check_out
FROM
    get_fit_now_check_in gt
INNER JOIN
    get_fit_now_member g
ON
    gt.membership_id = g.id
where g.membership_status = "gold" and gt.check_out_time >= 1600;
'''
pd.read_sql_query(query8, conn)


,gym_id,person_id,name,status,date,check_in,check_out
0,48Z7A,28819,Joe Germuska,gold,20180109,1600,1730
1,48Z55,67318,Jeremy Bowers,gold,20180109,1530,1700
2,90081,16371,Annabel Miller,gold,20180109,1600,1700


Combining the two witness Anaylsis.

witnes 1 points:
1. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags.
2. The man got into a car with a plate that included "H42W".

witness 2 points:  
1. He saw the muder at the time last week January the 9th.
2. He checked in at 1600 (4 clock) and checked out at 1700 (5 clock) on January the 9th.

Below is the murderer Name


In [ ]:
query9 = '''SELECT
    gt.membership_id AS gym_id,
    g.person_id,
    g.name,
    p.license_id AS license,
    d.plate_number,
    d.car_make,
    d.car_model,
    g.membership_status AS status,
    gt.check_in_date AS date,
    gt.check_in_time AS check_in,
    gt.check_out_time AS check_out
FROM
    get_fit_now_check_in gt
INNER JOIN
    get_fit_now_member g ON gt.membership_id = g.id
INNER JOIN
    person p ON g.person_id = p.id
INNER JOIN
    drivers_license d ON p.license_id = d.id
WHERE
    g.membership_status = 'gold'
    AND gt.check_out_time >= 1700
    AND d.plate_number LIKE '%H42W%';
'''
pd.read_sql_query(query9, conn)

,gym_id,person_id,name,license,plate_number,car_make,car_model,status,date,check_in,check_out
0,48Z55,67318,Jeremy Bowers,423327,0H42W2,Chevrolet,Spark LS,gold,20180109,1530,1700


In [ ]:
# inserting the murderer name in the solution table
query = '''
INSERT INTO solution VALUES(1, 'Jeremy Bowers');
'''

cursor = conn.cursor()
cursor.execute(query)
conn.commit()
print("Insertion successful!")

cursor.close()

query10 = '''
SELECT * FROM solution;
'''

pd.read_sql_query(query10, conn)

Insertion successful!


,user,value
0,0,"Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."


Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer.

From above the following are the points:  
1. Interview the murderer : Jeremy Bowers

In [ ]:
query11 = '''
SELECT *
FROM interview
WHERE person_id = 67318;
'''

pd.read_sql_query(query11, conn)

,person_id,transcript
0,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


After Investigating Jeremy Bowers From above points to be noted is
1. Other Murderer is Woemen.
2. she's around 5'5" (65") or 5'7" (67").
3. She has red hair
4. she drives a Tesla Model S.
5. I know that she attended the SQL Symphony Concert 3 times in December 2017

In [ ]:
# Here is the second cirminial
query12 = '''
SELECT
    f.person_id,
    p.name,
    d.height,
    d.hair_color,
    d.gender,
    d.car_make,
    d.car_model,
    f.event_name,
    COUNT(f.event_name) AS count,
    f.date
FROM
    facebook_event_checkin f
INNER JOIN
    person p ON f.person_id = p.id
INNER JOIN
    drivers_license d ON p.license_id = d.id
WHERE
    d.gender = 'female' AND
    d.height BETWEEN 65 AND 75 AND
    d.hair_color = 'red' AND
    d.car_make = 'Tesla' AND
    d.car_model = 'Model S' AND
    f.date LIKE '201712__';
'''

pd.read_sql_query(query12, conn)


,person_id,name,height,hair_color,gender,car_make,car_model,event_name,count,date
0,99716,Miranda Priestly,66,red,female,Tesla,Model S,SQL Symphony Concert,3,20171206


In [ ]:
# inserting the above record into solutons
query12 = '''
INSERT INTO solution VALUES(1, 'Miranda Priestly'), (2, 'Jeremy Bowers');
'''

cursor = conn.cursor()
cursor.execute(query12)
conn.commit()
cursor.close()
print("Inserted Record")

query13 = '''
SELECT * FROM solution;
'''
pd.read_sql_query(query13, conn)

Inserted Record


,user,value
0,0,"Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!"
1,2,Jeremy Bowers
